In [12]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models
import numpy as np
import os
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras import layers


First thing first we need to define some of the constraints of the problem.

First off there are only 2 classes (either cat and dog).

We can use the sigmoid activation function since we only have two classes

The pictures are in color (RGB)

We also have two folders one for cats and the other for dogs.

This means when we make the full dataset, we will have to turn all the pictures into arrays. ensure we keep the labels.
Then we need to make sure that all the cats and dogs datasets can be combined and that you choose the data randomly (with test_train_split)



In [ ]:
data_file_path = 'C:/Users/franc/Documents/neural_nets/resnet_cats_vs_dogs/PetImages'

In [7]:
train_dataset = keras.preprocessing.image_dataset_from_directory(
    data_file_path,
    labels='inferred',
    label_mode='categorical',
    batch_size = 64,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='training',
    image_size=(224,224)
)

validation_dataset = keras.preprocessing.image_dataset_from_directory(
    data_file_path,
    labels='inferred',
    label_mode = 'categorical',
    batch_size = 64,
    shuffle = True,
    seed = 42,
    validation_split = 0.2,
    subset = 'validation',
    image_size = (224,224)

)


Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


I will now make the train and validation split. 

Kindly remember that this will be an image pipeline not a dataframe pipeline since turning the data into an df pipeline would flatten the data

In [9]:
type(validation_dataset)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

In [11]:
autotune = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=autotune)
validation_dataset = validation_dataset.prefetch(buffer_size=autotune)

I then want to do some augmentation on the train part of the data. 

So that the finetuning can be more effective

In [13]:
data_augment = keras.Sequential([
    layers.RandomRotation(0.05),        # ~±9°
    layers.RandomZoom(0.1),
    layers.RandomFlip("horizontal"),
    layers.RandomContrast(0.1),
])

# Apply during training only:
# augmented_train_ds = train_dataset.map(lambda x, y: (data_augment(x, training=True), y))
# augmented_train_ds = augmented_train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

In [14]:
#Apply during training only:
augmented_train_ds = train_dataset.map(lambda x,y:(data_augment(x,training=True),y))

Importing the Resnet50 model

The reason why I chose Resnet50 is because it can be imported through tensorflow unlike some other smaller variants of resnet that can only be used with the transformers library.

In [15]:
base_model = keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step


I will first attempt to use the frozen model to make predictions using the pictures that we have